In [1]:
import os 
import json
import time
import glob
import numpy as np
import pandas as pd
import datetime
import re
from collections import Counter

import matplotlib.pyplot as plt

import spacy

nlp = spacy.load('en_core_web_sm')

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


In [2]:
# from rake_nltk import Rake

# r = Rake()

# '''
#     https://github.com/csurfer/rake-nltk
    
#     https://github.com/aneesha/RAKE
# '''


In [3]:
training_data = pd.read_csv('Stock market project - all training samples.csv')
training_data.shape

(575, 10)

In [4]:
training_data.head()

,dates,symbol,name,open price,1st price change,% change,time of 1st price change,published,news,label (p/n)
0,2020/10/02,HSBC,HSBC Holdings PLC,$19.51,$19.91,2.05%,2020-10-02T23:41:38,"Fri, 02 Oct 2020 18:00:11 GMT",HSBC Holdings plc (NYSE:HSBC) Currently 99.45%...,n
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Fri, 02 Oct 2020 17:19:17 GMT",Oman Cards and Payments Market Report- Growth ...,n
2,2020/10/02,LSCC,Lattice Semiconductor Corp,$29.75,$28.09,-5.58%,2020-10-03T00:10:57,-,-,NaN
3,2020/10/05,AGIO,Agios Pharmaceuticals Inc,$33.42,$36.38,8.86%,2020-10-06T00:11:38,"Mon, 05 Oct 2020 18:29:31 GMT","Agios Pharmaceuticals Inc (AGIO) Stock: Buy, H...",n
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Mon, 05 Oct 2020 17:18:00 GMT",Should be in your portfolio? Its Depends: Alle...,n


In [5]:
'''
    Complete the training dataset:
'''

dat = training_data.copy()

for i in range(1, dat.shape[0]):
    if pd.isnull(dat.iloc[i, 0]): # position 
        dat.iloc[i, :7] = dat.iloc[i-1, :7]

'''
    news_source_list: list of website names 

'''

dat['news_source'] = ''
dat['news_wo_source'] = ''

news_source_list = []

for i in range(dat.shape[0]):
    tmp_news = dat.loc[i, 'news']
    tmp_news_list = tmp_news.split(' - ')
    if len(tmp_news_list) > 1:
        dat.loc[i, 'news_wo_source'] = ' - '.join(tmp_news_list[:-1])
        dat.loc[i, 'news_source'] = tmp_news_list[-1]
        if tmp_news_list[-1] not in news_source_list:
            news_source_list.append(tmp_news_list[-1])
    else:
        dat.loc[i, 'news_wo_source'] = tmp_news

len(news_source_list)

210

In [28]:
news_source_list[:10]

['Marketing Sentinel',
 'The Market Records',
 'InvestorsObserver',
 'BOV News',
 'Nasdaq',
 'Investing.com',
 'The InvestChronicle',
 'The Oracle Dispatch',
 'Simply Wall St',
 'Seeking Alpha']

In [6]:
dat.head()

,dates,symbol,name,open price,1st price change,% change,time of 1st price change,published,news,label (p/n),news_source,news_wo_source
0,2020/10/02,HSBC,HSBC Holdings PLC,$19.51,$19.91,2.05%,2020-10-02T23:41:38,"Fri, 02 Oct 2020 18:00:11 GMT",HSBC Holdings plc (NYSE:HSBC) Currently 99.45%...,n,Marketing Sentinel,HSBC Holdings plc (NYSE:HSBC) Currently 99.45%...
1,2020/10/02,HSBC,HSBC Holdings PLC,$19.51,$19.91,2.05%,2020-10-02T23:41:38,"Fri, 02 Oct 2020 17:19:17 GMT",Oman Cards and Payments Market Report- Growth ...,n,The Market Records,Oman Cards and Payments Market Report- Growth ...
2,2020/10/02,LSCC,Lattice Semiconductor Corp,$29.75,$28.09,-5.58%,2020-10-03T00:10:57,-,-,NaN,,-
3,2020/10/05,AGIO,Agios Pharmaceuticals Inc,$33.42,$36.38,8.86%,2020-10-06T00:11:38,"Mon, 05 Oct 2020 18:29:31 GMT","Agios Pharmaceuticals Inc (AGIO) Stock: Buy, H...",n,InvestorsObserver,"Agios Pharmaceuticals Inc (AGIO) Stock: Buy, H..."
4,2020/10/05,AGIO,Agios Pharmaceuticals Inc,$33.42,$36.38,8.86%,2020-10-06T00:11:38,"Mon, 05 Oct 2020 17:18:00 GMT",Should be in your portfolio? Its Depends: Alle...,n,BOV News,Should be in your portfolio? Its Depends: Alle...


In [7]:
dat_up = dat[dat['% change'].map(lambda x: float(x.split('%')[0]) > 0)].reset_index(drop=True)
dat_down = dat[dat['% change'].map(lambda x: float(x.split('%')[0]) < 0)].reset_index(drop=True)

print(dat.shape, dat_up.shape, dat_down.shape)



(575, 12) (362, 12) (213, 12)


In [8]:
dat_up.head()

,dates,symbol,name,open price,1st price change,% change,time of 1st price change,published,news,label (p/n),news_source,news_wo_source
0,2020/10/02,HSBC,HSBC Holdings PLC,$19.51,$19.91,2.05%,2020-10-02T23:41:38,"Fri, 02 Oct 2020 18:00:11 GMT",HSBC Holdings plc (NYSE:HSBC) Currently 99.45%...,n,Marketing Sentinel,HSBC Holdings plc (NYSE:HSBC) Currently 99.45%...
1,2020/10/02,HSBC,HSBC Holdings PLC,$19.51,$19.91,2.05%,2020-10-02T23:41:38,"Fri, 02 Oct 2020 17:19:17 GMT",Oman Cards and Payments Market Report- Growth ...,n,The Market Records,Oman Cards and Payments Market Report- Growth ...
2,2020/10/05,AGIO,Agios Pharmaceuticals Inc,$33.42,$36.38,8.86%,2020-10-06T00:11:38,"Mon, 05 Oct 2020 18:29:31 GMT","Agios Pharmaceuticals Inc (AGIO) Stock: Buy, H...",n,InvestorsObserver,"Agios Pharmaceuticals Inc (AGIO) Stock: Buy, H..."
3,2020/10/05,AGIO,Agios Pharmaceuticals Inc,$33.42,$36.38,8.86%,2020-10-06T00:11:38,"Mon, 05 Oct 2020 17:18:00 GMT",Should be in your portfolio? Its Depends: Alle...,n,BOV News,Should be in your portfolio? Its Depends: Alle...
4,2020/10/05,AGIO,Agios Pharmaceuticals Inc,$33.42,$36.38,8.86%,2020-10-06T00:11:38,"Mon, 05 Oct 2020 16:00:00 GMT",All You Need to Know About Agios Pharmaceutica...,n,Nasdaq,All You Need to Know About Agios Pharmaceutica...


In [9]:
dat_up.groupby(['dates', 'symbol']).agg({'dates': len}).head()

dates
dates      symbol       
2020/10/02 HSBC        2
2020/10/05 AGIO        3
           AQN         1
           LSCC        1
           MRVL        4

In [10]:
dat['name'].unique()

array(['HSBC Holdings PLC', 'Lattice Semiconductor Corp',
       'Agios Pharmaceuticals Inc', 'Algonquin Power & Utilities Corp',
       'EPR Properties', 'Marvell Technology Group Ltd', 'MyoKardia Inc',
       'Regeneron Pharmaceuticals Inc', 'Wipro Ltd',
       'Bristol-Myers Squibb Co', 'Arcelormittal SA', 'Netflix Inc',
       'NovoCure Ltd', 'Quidel Corp', 'Sirius XM Holdings Inc',
       'Sarepta Therapeutics Inc', 'United Microelectronics Corp',
       'Qiagen NV', 'Parsley Energy Inc', 'Trade Desk Inc (The)',
       'Twist Bioscience Corp', 'J.B. Hunt Transport Services Inc.',
       'Teradata Corp', 'Comerica Inc', 'MercadoLibre Inc',
       'Conocophillips', 'Casella Waste Systems Inc',
       'Fiverr International Ltd', 'Pinterest Inc', 'PayPal Holdings Inc',
       'Teradyne Inc.', 'Trupanion Inc', 'Twitter Inc',
       'Align Technology Inc', 'Crown Castle International Corp',
       'Huntington Bancshares Inc', 'JetBlue Airways Corp',
       'PulteGroup Inc', 'TRI Pointe 

In [11]:
'''
    Stocks_key_words_dict
'''
# del_words = ['Ltd', 'Inc', 'Corp', 'PLC', 'LLC', 'Corporation', 
#              'HOLDERs', 'Co', 'Limited', 'Company',
#              'Providers', 'group', 'com', 'Brands', 
#              'NA', 'SA', 'NV', 's', 'a', 'and']
# del_words = [w.lower() for w in del_words]

# stocks_key_words_dict = {}

# for s in list(sorted(dat['symbol'].unique())):
#     w_lemmatizer = WordNetLemmatizer()

#     tmp_words = []
#     tmp_words.append(s.lower())
#     tmp_names = [w.lower() for w in re.findall(r'\w+', dat[dat['symbol'] == s]['name'].values[0]) \
#                  if w.lower() not in del_words]
#     tmp_words.extend(tmp_names)
#     tmp_words.append(' '.join(tmp_names))
    
#     tmp_words = list(set([w for w in tmp_words]))
#     stocks_key_words_dict[s] = tmp_words
    

'\n    Stocks_key_words_dict\n'

In [12]:
'''
    stocks_NameSymbolStr_dict
'''

stocks_NameSymbolStr_dict = {}

for s in list(sorted(dat['symbol'].unique())):
    tmp_NameStr = dat[dat['symbol'] == s]['name'].values[0]
    stocks_NameSymbolStr_dict[s] = ' '.join([s, tmp_NameStr])
    

In [42]:

def check_word_in_sent(word, sentence):
    # lowercase
    # words: list of word
    # sentence: string - stocks_NameSymbolStr_dict
    
    # all_words = re.findall(r'\w+', sentence)
    all_words = [t.text.lower() for t in nlp(sentence)]
    if word.lower() in all_words:
#         print('{}, Found!'.format(word))
        return True
    else:
#         print('{}, Not in sentence!'.format(word))
        return False


In [16]:
stocks_NameSymbolStr_dict['ZION'], \
check_word_in_sent('zion', stocks_NameSymbolStr_dict['ZION'])

zion, Found!


('ZION Zions Bancorporation NA', True)

### The necessary steps include (but aren’t limited to) the following:

1. Tokenizing sentences to break text down into sentences, words, or other units
2. Removing stop words like “if,” “but,” “or,” and so on


In [61]:
del_words = ['$', '|', 'inc.', 'inc', 'corp', 'corporation', 'company', 'co', 
             'stock', 'stocks', 'nyse', 'nasdaq', 'llc', 'ltd.', 'limited', 'ltd',
             'com', 'brands', 'market']

In [62]:
'''
    News data processing:
    - filter out:
        - company name and symbol
        - news sources: website name
        - stopwords
        - delimiters
        
'''

keywords_up_dict = {}
keywords_up_list = []

for i in range(dat_up.shape[0]):
    s = dat_up.loc[i, 'symbol']

    tmp_news = dat_up.loc[i, 'news_wo_source']

    tmp_keywords = [(token.text.lower(), token.lemma_.lower(), token.pos_) for token in nlp(tmp_news) 
                     if not token.is_stop
                     if not token.is_punct
                     if not token.is_space
                     if not check_word_in_sent(token.text, stocks_NameSymbolStr_dict[s])
                     if token.text.lower() not in del_words]

    tmp_keywords
    if s not in keywords_up_dict:
        keywords_up_dict[s] = []

    keywords_up_dict[s].extend(tmp_keywords)

    keywords_up_list.extend(tmp_keywords)


In [63]:
len(dat_up['symbol'].unique()), len(keywords_up_dict.keys()), len(keywords_up_list)

(61, 61, 2549)

In [64]:
keywords_up_list[:10]

[('currently', 'currently', 'ADV'),
 ('99.45', '99.45', 'NUM'),
 ('52-week', '52-week', 'NUM'),
 ('high', 'high', 'ADJ'),
 ('upside', 'upside', 'PROPN'),
 ('potential', 'potential', 'PROPN'),
 ('surprise', 'surprise', 'VERB'),
 ('oman', 'oman', 'PROPN'),
 ('cards', 'cards', 'PROPN'),
 ('payments', 'payments', 'PROPN')]

In [70]:
np.unique([k[2] for k in keywords_up_list])

array(['ADJ', 'ADP', 'ADV', 'CCONJ', 'NOUN', 'NUM', 'PROPN', 'SCONJ',
       'VERB', 'X'], dtype='<U5')

In [67]:
Counter([k[1] for k in keywords_up_list if k[2] == 'VERB']).most_common(20)

[('buy', 22),
 ('sell', 13),
 ('announce', 11),
 ('rise', 11),
 ('beat', 11),
 ('get', 9),
 ('hold', 6),
 ('investigate', 6),
 ('lead', 6),
 ('say', 6),
 ('hit', 6),
 ('acquire', 5),
 ('surge', 5),
 ('let', 5),
 ('add', 5),
 ('encourage', 4),
 ('contact', 4),
 ('propose', 4),
 ('gain', 4),
 ('jump', 4)]

In [ ]:
Counter([k[1] for k in keywords_up_list if k[2] == 'VERB']).most_common(20)